In [1]:
import json
from pathlib import Path
import pandas as pd 

In [ ]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-mini-temperature-sweep/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

422 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/musique',
   'name': 'answerable',
   'split': 'validation'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.1,
   'system_prompt': 'no-role.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'empty.json'},
  'run': 1},
 'metrics': {}}

In [ ]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-mini")
    & (df["params.qa.system_prompt"] != "no-role.txt")
    & (df["params.qa.temperature"] > 0.0)
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

412 experiments after preprocessing


In [4]:
def parse_qa_technique(system_prompt_filename):
    if "cte" in system_prompt_filename:
        return "CTE"
    if "cot" in system_prompt_filename:
        return "COT"
    return "Standard"

df["params.qa.technique"] = df["params.qa.system_prompt"].map(parse_qa_technique)

In [5]:
import re

def parse_n_shot(x):
    if 'empty' in x:
        return 0
    # match the digit in the string
    x = re.search(r'\d+', x).group()
    return int(x)

assert parse_n_shot('empty.json') == 0
assert parse_n_shot('cot-2-shot.json') == 2

In [6]:
df['params.qa.n_shot'] = df['params.qa.few_shot_examples'].map(parse_n_shot)

In [7]:
def sorted_tuple(x):
    return tuple(sorted(x))

run_counts = (
    df.groupby(["params.qa.system_prompt", "params.qa.user_prompt_template", "params.qa.few_shot_examples", "params.qa.temperature"])[
        "params.run"
    ]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) < 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.temperature,params.run
0,cot/excellent-format-reasoning.txt,cq.txt,cot-2-shot.json,0.1,"(1,)"
1,cot/excellent-format-reasoning.txt,cq.txt,cot-2-shot.json,0.3,"(1,)"
2,cot/excellent-format-reasoning.txt,cq.txt,cot-2-shot.json,0.5,"(1,)"
3,cot/excellent-format-reasoning.txt,cq.txt,cot-2-shot.json,0.7,"(1,)"
4,cot/excellent-format-reasoning.txt,cq.txt,cot-2-shot.json,1.0,"(1,)"
5,cot/excellent-format-reasoning.txt,cq.txt,cot-2-shot.json,1.5,"(1,)"
6,cot/excellent-format-reasoning.txt,cq.txt,cot-2-shot.json,2.0,"(1,)"
7,cot/excellent-format-reasoning.txt,cq.txt,empty.json,0.1,"(1,)"
8,cot/excellent-format-reasoning.txt,cq.txt,empty.json,0.3,"(1,)"
9,cot/excellent-format-reasoning.txt,cq.txt,empty.json,0.5,"(1,)"


In [8]:
run_counts.loc[run_counts["params.run"].map(len) > 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.temperature,params.run


In [9]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	cot/excellent-format-reasoning.txt
	cot/format-minimal.txt
	cot/format-reasoning.txt
	cot/format-think.txt
	cte/excellent-format-few.txt
	cte/format-few.txt
	cte/format-relevant-few.txt
	standard/excellent-few.txt
	standard/helpful-output-format-few.txt
	standard/minimal-output-format-few.txt
	standard/minimal-output-format.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	cot-2-shot.json
	empty.json
	cte-2-shot-delim-1.json
	cte-2-shot-delim-2.json
	standard-2-shot.json
- params.qa.temperature
	0.1
	0.3
	0.5
	0.7
	1.0
	1.5
	2.0
- params.run
	(1,)
	(1, 2, 3)


In [10]:
df.to_json('results.jsonl', orient='records', lines=True)